<a href="https://colab.research.google.com/github/Sid-Oya/DS-Unit-1-Sprint-3-Linear-Algebra/blob/master/Copy_of_DS_134_Clustering_Assignment_of_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 0. Principle Components Analysis Review

### 0.1 Notes about PCA

**Principal Components are not a re-labeling of the original features**

I saw some confusion yesterday about what the new Principal Components are that come out of our PCA transformations. Principal Components are a linear combination of any and all dimensions (features) that will increase their variance, this means that PCs are made up of a mixture of features --mostly the ones with the highest variance, but also smaller parts from other features. This means that they are not comparable to the original features of our $X$ matrix. In cases where we're not reducing dimensionality that much (like the Iris dataset) our Principal Components might be extremely similar to the original features (since there's not that many to pull from) but don't think of them in that way, think of them as a completely new dataset that we can't really apply 

**PCA does not make predictions**

I would not call PCA a "machine learning algorithm" in that it does not try to make any predictions. We can't calculate any accuracy measure. You can call it an algorithm, you can call it a preprocessing technique or method, but it's not truly making predictions. This may have been confusing due to the fact that the Iris dataset had labels, but PCA is just re-organizing points in space, it's not making any predictions.

**PCA does not standardize the data for you**

You'll notice in the "from scratch" implementation of PCA that I did in class yesterday that in that example I did not divide the points by the standard deviation. I believe you'll get a slightly different set of points if you choose to divide by the standard deviation (I think this might be what A Apte was seeing yesterday when he tried both methods and found that they looked different. It could be something else entirely, but that's my first guess at what could be going on.)

The Sklearn implementation does not standardize the points for you as part of the process. You can either do this yourself "by hand" or you can use other sklearn methods like this preprocessing step which will automatically standardize your data to have a mean of 0 and a standard deviation of 1. You have to do this **before** you pass your data to PCA.

<https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html>

**PCA does not retain 100% of the information of the original dataset**

Each component explains a certain % of the variance of the original dataset. PCA tries to maximize that variance, but you might need to use more than 2 components. 

Typically you want to use enough components in your analysis to keep the explained variance > 90%.

So we're trading off losing a small-medium amount of predictive power for a reduction in dimensions/size.

## 0.2 Housing Affordability Data System (HADS) Example

The Housing Affordability Data System (HADS) is a set of files derived from the 1985 and later national American Housing Survey (AHS)


In [0]:
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read Natinal Data 
national_url = 'https://www.huduser.gov/portal/datasets/hads/hads2013n_ASCII.zip'
national_file = 'thads2013n.txt'

if os.path.exists(national_file):
    national = pd.read_csv(national_file)
else: 
    z_national = urlopen(national_url)
    zip_national = ZipFile(BytesIO(z_national.read())).extract(national_file)
    national = pd.read_csv(zip_national)

national.head()

,CONTROL,AGE1,METRO3,REGION,LMED,FMR,L30,L50,L80,IPOV,BEDRMS,BUILT,STATUS,TYPE,VALUE,VACANCY,TENURE,NUNITS,ROOMS,WEIGHT,PER,ZINC2,ZADEQ,ZSMHC,STRUCTURETYPE,OWNRENT,UTILITY,OTHERCOST,COST06,COST12,COST08,COSTMED,TOTSAL,ASSISTED,GLMED,GL30,GL50,GL80,APLMED,ABL30,...,COST08RELPOVCAT,COST08RELFMRPCT,COST08RELFMRCAT,COST12RELAMIPCT,COST12RELAMICAT,COST12RELPOVPCT,COST12RELPOVCAT,COST12RELFMRPCT,COST12RELFMRCAT,COSTMedRELAMIPCT,COSTMedRELAMICAT,COSTMedRELPOVPCT,COSTMedRELPOVCAT,COSTMedRELFMRPCT,COSTMedRELFMRCAT,FMTZADEQ,FMTMETRO3,FMTBUILT,FMTSTRUCTURETYPE,FMTBEDRMS,FMTOWNRENT,FMTCOST06RELPOVCAT,FMTCOST08RELPOVCAT,FMTCOST12RELPOVCAT,FMTCOSTMEDRELPOVCAT,FMTINCRELPOVCAT,FMTCOST06RELFMRCAT,FMTCOST08RELFMRCAT,FMTCOST12RELFMRCAT,FMTCOSTMEDRELFMRCAT,FMTINCRELFMRCAT,FMTCOST06RELAMICAT,FMTCOST08RELAMICAT,FMTCOST12RELAMICAT,FMTCOSTMEDRELAMICAT,FMTINCRELAMICAT,FMTASSISTED,FMTBURDEN,FMTREGION,FMTSTATUS
0,'100003130103',82,'3','1',73738,956,15738,26213,40322,11067,2,2006,'1',1,40000,-6,'1',1,6,3117.394239,1,18021,'1',533,1,'1',169.000000,213.750000,648.588189,803.050535,696.905247,615.156712,0,-9,73738,15738,26213,40322,51616.6,20234.571429,...,4,72.898038,2,48.402635,2,290.250487,4,84.001102,2,37.077624,2,222.339102,4,64.346936,2,'1 Adequate','-5','2000-2009','1 Single Family','2 2BR','1 Owner','4 200%+ Poverty','4 200%+ Poverty','4 200%+ Poverty','4 200%+ Poverty','3 150-200% Poverty','2 50.1 - 100% FMR','2 50.1 - 100% FMR','2 50.1 - 100% FMR','2 50.1 - 100% FMR','1 LTE 50% FMR','2 30 - 50% AMI','2 30 - 50% AMI','2 30 - 50% AMI','2 30 - 50% AMI','2 30 - 50% AMI','.','2 30% to 50%','-5','-5'
1,'100006110249',50,'5','3',55846,1100,17165,28604,45744,24218,4,1980,'1',1,130000,-6,'1',1,6,2150.725544,4,122961,'1',487,1,'1',245.333333,58.333333,1167.640781,1669.643405,1324.671218,1058.988479,123000,-9,55846,17165,28604,45744,55846.0,19911.400000,...,4,120.424656,3,103.094063,6,275.768999,4,151.785764,3,65.388468,4,174.909320,3,96.271680,2,'1 Adequate','-5','1980-1989','1 Single Family','4 4BR+','1 Owner','3 150-200% Poverty','4 200%+ Poverty','4 200%+ Poverty','3 150-200% Poverty','4 200%+ Poverty','3 GT FMR','3 GT FMR','3 GT FMR','2 50.1 - 100% FMR','3 GT FMR','4 60 - 80% AMI','4 60 - 80% AMI','6 100 - 120% AMI','4 60 - 80% AMI','7 120% AMI +','.','1 Less than 30%','-5','-5'
2,'100006370140',53,'5','3',55846,1100,13750,22897,36614,15470,4,1985,'1',1,150000,-6,'1',1,7,2213.789404,2,27974,'1',1405,1,'1',159.000000,37.500000,1193.393209,1772.627006,1374.582175,1068.025168,28000,-9,55846,13750,22897,36614,44676.8,19937.500000,...,4,124.962016,3,109.452905,6,458.339239,4,161.147910,3,65.946449,4,276.153890,4,97.093197,2,'1 Adequate','-5','1980-1989','1 Single Family','4 4BR+','1 Owner','4 200%+ Poverty','4 200%+ Poverty','4 200%+ Poverty','4 200%+ Poverty','3 150-200% Poverty','3 GT FMR','3 GT FMR','3 GT FMR','2 50.1 - 100% FMR','2 50.1 - 100% FMR','4 60 - 80% AMI','5 80 - 100% AMI','6 100 - 120% AMI','4 60 - 80% AMI','4 60 - 80% AMI','.','3 50% or More','-5','-5'
3,'100006520140',67,'5','3',55846,949,13750,22897,36614,13964,3,1985,'1',1,200000,-6,'1',1,6,2364.585097,2,32220,'1',279,1,'1',179.000000,70.666667,1578.857612,2351.169341,1820.442900,1411.700224,0,-9,55846,13750,22897,36614,44676.8,17875.000000,...,4,191.827492,3,161.926709,7,673.494512,4,247.752301,3,97.224801,5,404.382763,4,148.756610,3,'1 Adequate','-5','1980-1989','1 Single Family','3 3BR','1 Owner','4 200%+ Poverty','4 200%+ Poverty','4 200%+ Poverty','4 200%+ Poverty','4 200%+ Poverty','3 GT FMR','3 GT FMR','3 GT FMR','3 GT FMR','2 50.1 - 100% FMR','6 100 - 120% AMI','7 120% AMI +','7 120% AMI +','5 80 - 100% AMI','4 60 - 80% AMI','.','1 Less than 30%','-5','-5'
4,'100007130148',26,'1','3',60991,737,14801,24628,39421,15492,2,1980,'1',1,-6,-6,'2',100,4,2314.524902,2,96874,'1',759,5,'2',146.000000,12.500000,759.000000,759.000000,759.000000,759.000000,96900,0,60991,14801,24628,39421,48792.8,16651.125000,...,3,102.985075,3,55.308707,3,195.972115,3,102.985075,3,55.308707,3,195.972115,3,102.98

In [0]:
national['METRO3'].value_counts().sort_index()

'1'    21493
'2'    24936
'3'     6851
'4'     4462
'5'     6793
Name: METRO3, dtype: int64

In [0]:
# one-hot encoding: it will create 5 new variables, each 0/1

In [0]:
national['met_values'] = national['METRO3'].map({'1':0, '2':3})

In [0]:
national.shape

(64535, 100)

In [0]:
national.corr().shape

(68, 68)

### 0.2.1 Preprocess all the categorical columns

In [0]:
# which columns are object? or numeric?
national.dtypes

CONTROL         object
AGE1             int64
METRO3          object
REGION          object
LMED             int64
                ...   
FMTASSISTED     object
FMTBURDEN       object
FMTREGION       object
FMTSTATUS       object
met_values     float64
Length: 100, dtype: object

In [0]:
# make lists of categorical and numeric columns
cat_cols = national.columns[national.dtypes=='object']
print(len(cat_cols))

32


In [0]:
# do the same for numeric
num_cols = national.columns[national.dtypes!='object']
print(len(num_cols))

68


In [0]:
# What are the cat columns?
cat_cols[:5]

Index(['CONTROL', 'METRO3', 'REGION', 'STATUS', 'TENURE'], dtype='object')

In [0]:
# how many do we have of each?
# did that.

In [0]:
# Cast all the categorical columns to "category" data type
# .astype()
# national['METRO3'].astype('category')
national[cat_cols] = national[cat_cols].astype('category')

In [0]:
# check my work
national['METRO3'].dtype

CategoricalDtype(categories=[''1'', ''2'', ''3'', ''4'', ''5''], ordered=False)

In [0]:
# Remove apostrophes from all of the categorical columns
for col in cat_cols:
  national[col] = national[col].cat.codes

In [0]:
national.head()

,CONTROL,AGE1,METRO3,REGION,LMED,FMR,L30,L50,L80,IPOV,BEDRMS,BUILT,STATUS,TYPE,VALUE,VACANCY,TENURE,NUNITS,ROOMS,WEIGHT,PER,ZINC2,ZADEQ,ZSMHC,STRUCTURETYPE,OWNRENT,UTILITY,OTHERCOST,COST06,COST12,COST08,COSTMED,TOTSAL,ASSISTED,GLMED,GL30,GL50,GL80,APLMED,ABL30,...,COST08RELFMRPCT,COST08RELFMRCAT,COST12RELAMIPCT,COST12RELAMICAT,COST12RELPOVPCT,COST12RELPOVCAT,COST12RELFMRPCT,COST12RELFMRCAT,COSTMedRELAMIPCT,COSTMedRELAMICAT,COSTMedRELPOVPCT,COSTMedRELPOVCAT,COSTMedRELFMRPCT,COSTMedRELFMRCAT,FMTZADEQ,FMTMETRO3,FMTBUILT,FMTSTRUCTURETYPE,FMTBEDRMS,FMTOWNRENT,FMTCOST06RELPOVCAT,FMTCOST08RELPOVCAT,FMTCOST12RELPOVCAT,FMTCOSTMEDRELPOVCAT,FMTINCRELPOVCAT,FMTCOST06RELFMRCAT,FMTCOST08RELFMRCAT,FMTCOST12RELFMRCAT,FMTCOSTMEDRELFMRCAT,FMTINCRELFMRCAT,FMTCOST06RELAMICAT,FMTCOST08RELAMICAT,FMTCOST12RELAMICAT,FMTCOSTMEDRELAMICAT,FMTINCRELAMICAT,FMTASSISTED,FMTBURDEN,FMTREGION,FMTSTATUS,met_values
0,0,82,2,0,73738,956,15738,26213,40322,11067,2,2006,0,1,40000,-6,1,1,6,3117.394239,1,18021,1,533,1,0,169.000000,213.750000,648.588189,803.050535,696.905247,615.156712,0,-9,73738,15738,26213,40322,51616.6,20234.571429,...,72.898038,2,48.402635,2,290.250487,4,84.001102,2,37.077624,2,222.339102,4,64.346936,2,1,0,5,1,2,0,4,4,4,4,3,1,1,1,1,1,1,1,1,1,2,0,2,0,0,NaN
1,1,50,4,2,55846,1100,17165,28604,45744,24218,4,1980,0,1,130000,-6,1,1,6,2150.725544,4,122961,1,487,1,0,245.333333,58.333333,1167.640781,1669.643405,1324.671218,1058.988479,123000,-9,55846,17165,28604,45744,55846.0,19911.400000,...,120.424656,3,103.094063,6,275.768999,4,151.785764,3,65.388468,4,174.909320,3,96.271680,2,1,0,3,1,4,0,3,4,4,3,4,2,2,2,1,3,3,3,5,3,7,0,1,0,0,NaN
2,2,53,4,2,55846,1100,13750,22897,36614,15470,4,1985,0,1,150000,-6,1,1,7,2213.789404,2,27974,1,1405,1,0,159.000000,37.500000,1193.393209,1772.627006,1374.582175,1068.025168,28000,-9,55846,13750,22897,36614,44676.8,19937.500000,...,124.962016,3,109.452905,6,458.339239,4,161.147910,3,65.946449,4,276.153890,4,97.093197,2,1,0,3,1,4,0,4,4,4,4,3,2,2,2,1,2,3,4,5,3,4,0,3,0,0,NaN
3,3,67,4,2,55846,949,13750,22897,36614,13964,3,1985,0,1,200000,-6,1,1,6,2364.585097,2,32220,1,279,1,0,179.000000,70.666667,1578.857612,2351.169341,1820.442900,1411.700224,0,-9,55846,13750,22897,36614,44676.8,17875.000000,...,191.827492,3,161.926709,7,673.494512,4,247.752301,3,97.224801,5,404.382763,4,148.756610,3,1,0,3,1,3,0,4,4,4,4,4,2,2,2,2,2,5,6,6,4,4,0,1,0,0,NaN
4,4,26,0,2,60991,737,14801,24628,39421,15492,2,1980,0,1,-6,-6,2,100,4,2314.524902,2,96874,1,759,5,1,146.000000,12.500000,759.000000,759.000000,759.000000,759.000000,96900,0,60991,14801,24628,39421,48792.8,16651.125000,...,102.985075,3,55.308707,3,195.972115,3,102.985075,3,55.308707,3,195.972115,3,102.985075,3,1,1,3,5,2,1,3,3,3,3,4,2,2,2,2,3,2,2,2,2,7,1,1,0,0,NaN


In [0]:
# How many numeric columns do we now have?
national.dtypes

CONTROL          int32
AGE1             int64
METRO3            int8
REGION            int8
LMED             int64
                ...   
FMTASSISTED       int8
FMTBURDEN         int8
FMTREGION         int8
FMTSTATUS         int8
met_values     float64
Length: 100, dtype: object

In [0]:
# count those
num_cols = national.columns[national.dtypes!='object']
print(national.shape)
len(num_cols)

(64535, 100)


100

### 0.2.2 Principal Components Analysis

In [0]:
# Make a copy of our dataframe, we will standarize the copy so as to not overwrite our original data
processed = national.copy()

In [0]:
# check it out
processed.head()

,CONTROL,AGE1,METRO3,REGION,LMED,FMR,L30,L50,L80,IPOV,BEDRMS,BUILT,STATUS,TYPE,VALUE,VACANCY,TENURE,NUNITS,ROOMS,WEIGHT,PER,ZINC2,ZADEQ,ZSMHC,STRUCTURETYPE,OWNRENT,UTILITY,OTHERCOST,COST06,COST12,COST08,COSTMED,TOTSAL,ASSISTED,GLMED,GL30,GL50,GL80,APLMED,ABL30,...,COST08RELFMRPCT,COST08RELFMRCAT,COST12RELAMIPCT,COST12RELAMICAT,COST12RELPOVPCT,COST12RELPOVCAT,COST12RELFMRPCT,COST12RELFMRCAT,COSTMedRELAMIPCT,COSTMedRELAMICAT,COSTMedRELPOVPCT,COSTMedRELPOVCAT,COSTMedRELFMRPCT,COSTMedRELFMRCAT,FMTZADEQ,FMTMETRO3,FMTBUILT,FMTSTRUCTURETYPE,FMTBEDRMS,FMTOWNRENT,FMTCOST06RELPOVCAT,FMTCOST08RELPOVCAT,FMTCOST12RELPOVCAT,FMTCOSTMEDRELPOVCAT,FMTINCRELPOVCAT,FMTCOST06RELFMRCAT,FMTCOST08RELFMRCAT,FMTCOST12RELFMRCAT,FMTCOSTMEDRELFMRCAT,FMTINCRELFMRCAT,FMTCOST06RELAMICAT,FMTCOST08RELAMICAT,FMTCOST12RELAMICAT,FMTCOSTMEDRELAMICAT,FMTINCRELAMICAT,FMTASSISTED,FMTBURDEN,FMTREGION,FMTSTATUS,met_values
0,0,82,2,0,73738,956,15738,26213,40322,11067,2,2006,0,1,40000,-6,1,1,6,3117.394239,1,18021,1,533,1,0,169.000000,213.750000,648.588189,803.050535,696.905247,615.156712,0,-9,73738,15738,26213,40322,51616.6,20234.571429,...,72.898038,2,48.402635,2,290.250487,4,84.001102,2,37.077624,2,222.339102,4,64.346936,2,1,0,5,1,2,0,4,4,4,4,3,1,1,1,1,1,1,1,1,1,2,0,2,0,0,NaN
1,1,50,4,2,55846,1100,17165,28604,45744,24218,4,1980,0,1,130000,-6,1,1,6,2150.725544,4,122961,1,487,1,0,245.333333,58.333333,1167.640781,1669.643405,1324.671218,1058.988479,123000,-9,55846,17165,28604,45744,55846.0,19911.400000,...,120.424656,3,103.094063,6,275.768999,4,151.785764,3,65.388468,4,174.909320,3,96.271680,2,1,0,3,1,4,0,3,4,4,3,4,2,2,2,1,3,3,3,5,3,7,0,1,0,0,NaN
2,2,53,4,2,55846,1100,13750,22897,36614,15470,4,1985,0,1,150000,-6,1,1,7,2213.789404,2,27974,1,1405,1,0,159.000000,37.500000,1193.393209,1772.627006,1374.582175,1068.025168,28000,-9,55846,13750,22897,36614,44676.8,19937.500000,...,124.962016,3,109.452905,6,458.339239,4,161.147910,3,65.946449,4,276.153890,4,97.093197,2,1,0,3,1,4,0,4,4,4,4,3,2,2,2,1,2,3,4,5,3,4,0,3,0,0,NaN
3,3,67,4,2,55846,949,13750,22897,36614,13964,3,1985,0,1,200000,-6,1,1,6,2364.585097,2,32220,1,279,1,0,179.000000,70.666667,1578.857612,2351.169341,1820.442900,1411.700224,0,-9,55846,13750,22897,36614,44676.8,17875.000000,...,191.827492,3,161.926709,7,673.494512,4,247.752301,3,97.224801,5,404.382763,4,148.756610,3,1,0,3,1,3,0,4,4,4,4,4,2,2,2,2,2,5,6,6,4,4,0,1,0,0,NaN
4,4,26,0,2,60991,737,14801,24628,39421,15492,2,1980,0,1,-6,-6,2,100,4,2314.524902,2,96874,1,759,5,1,146.000000,12.500000,759.000000,759.000000,759.000000,759.000000,96900,0,60991,14801,24628,39421,48792.8,16651.125000,...,102.985075,3,55.308707,3,195.972115,3,102.985075,3,55.308707,3,195.972115,3,102.985075,3,1,1,3,5,2,1,3,3,3,3,4,2,2,2,2,3,2,2,2,2,7,1,1,0,0,NaN


In [0]:
# call the instance
# instantiate the SKLearn class for standardization
# mean 0, std 1
# puts all features on the same metric!
from sklearn.preprocessing import StandardScaler
myscaler = StandardScaler()

In [0]:
# Standardize the dataset (default is normalization)
processed = myscaler.fit_transform(processed)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/extmath.py:765: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.6/dist-packages/sklearn/utils/extmath.py:706: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


In [0]:
# did that do what we want?
pd.DataFrame(processed).shape

(64535, 100)

In [0]:
# import and instantiate the PCA class
from sklearn.decomposition import PCA
mypca = PCA(n_components=10)

In [0]:
# Apply PCA to the data
pca_features = mypca.fit_transform(processed)

ValueError: ignored

In [0]:
# check our work
pd.DataFrame(pca_features).shape

In [0]:
# this is what I would use:
pd.DataFrame(pca_features).head()

In [0]:
#  how much variation did each principal component explain?
mypca.explained_variance_ratio_

In [0]:
# How much total variance did we explain?
sum(mypca.explained_variance_ratio_)

In [0]:
# How much information did we lose?
1-sum(mypca.explained_variance_ratio_)

### 0.2.3 Make a scree plot

In [0]:
# define the number of components
num_components = len(mypca.explained_variance_ratio_)
num_components

In [0]:
# create a numpy array as long as the number of components
index = np.arange(num_components)
index

In [0]:
# create an array of 10 values
vals = mypca.explained_variance_ratio_
vals

In [0]:
# Plot the variance explained by each component.
plt.bar(index, vals);

In [0]:
# Plot the cumulative variance explained by all the components.
import matplotlib.pyplot as plt

In [0]:
# Define scree plot function
def scree_plot():

    plt.figure(figsize=(18, 6))
    ax = plt.subplot(111)
   
    ax.xaxis.set_tick_params(width=0)
    ax.yaxis.set_tick_params(width=2, length=12)

    ax.set_xlabel("Principal Component")
    ax.set_ylabel("Variance Explained (%)")
    plt.title('Explained Variance Per Principal Component')
    plt.show()

In [0]:
# Apply the plot function to our principle component
scree_plot()

# Part 1. Intro to Clustering

### 2.1 Machine Learning Overview

How do you know what kind of Machine Learning that you're doing? What algorithm should you pick? 

This decision is driven by:

1) The attributes of your dataset

2) What you want to predict

__Supervised Learning__  

  - Classification algorithms try to predict the correct category (or class) from a given set of categories.
  - Regression algorithms predict a continuous or semi-continuous value. (Not to be confused with _Linear_ Regression)

__Unsupervised Learning__
  - Clustering
  Identifying groupings of related observations. This is our topic for today!
  - Dimensionality Reduction
  Takes a high-dimensionality dataset and reduces the number of variables taken into consideration via methods of feature selection and feature extraction.
  - Association Rule Learning
  Association is a method of discovering relationships between observations in a dataset. (between ovservations or features, not just relationships between explanatory variables and a single output variable. )

__Reinforcement Learning__  
* A form of machine learning where an "agent" interacts with its environment and is rewarded for correct behavior and penalized for incorrect behavior. 
* Over many iterations the agent learns the behavior that results in the greatest reward and smallest punishment. 

Memorize This!

**Supervised**: Labelled outputs
- **Classification**: Discrete output cagetories
- **Regression**: Continuous output values

**Unsupervised**: Outputs are not labelled

**Reinforcement**: Rewards/punishments for "behaviors"

Kaggle datasets:
training data has labels but the testing data does not.

[Classification Examples](https://github.com/ShuaiW/kaggle-classification)

[Regression Examples](https://github.com/ShuaiW/kaggle-classification)

[Unsupervised Learning Examples](http://www.lsi.upc.edu/~bejar/apren/docum/trans/09-clusterej-eng.pdf)

ML Cheat Sheets  
<div>
<img https://docs.microsoft.com/en-us/azure/machine-learning/studio/media/algorithm-cheat-sheet/machine-learning-algorithm-cheat-sheet-small_v_0_6-01.png width='200' />
</div>


![Microsoft Cheat Sheet](https://docs.microsoft.com/en-us/azure/machine-learning/studio/media/algorithm-cheat-sheet/machine-learning-algorithm-cheat-sheet-small_v_0_6-01.png)

This one does not group them by supervised, unsupervised, regression, classification, etc. But it gives you an idea of the different families of algorithms.

![Algorithm Map](https://jixta.files.wordpress.com/2015/11/machinelearningalgorithms.png?w=816&h=521&zoom=2)


### 1.2 No Free Lunch Principle

The no free lunch principle states that the more an algorithm is optimized to solve one specific kind of problem, the worse it gets at solving all other kinds of problems. 

This means that if you want an algorithm that's really good at solving a certain problem (cluster shape for example), it usually lose some of its ability to generalize to other problems. 

**What does this mean for us as data scientists?**

1) There are always tradeoffs when selecting from different approaches. Because of this, understanding those tradeoffs and justifying your choice of methodology is just as important as actually doing the work itself.

2) The only way that we can choose one approach over another is to make assumptions about our data. If we don't know anything about the characteristics of our data, then we can't make an informed choice of algorithm. 

Think about how we knew to use Unsupervised vs Supervised learning for the clustering problem, the choice was informed by our data. Does it have labels or not? 

![No Free Lunch](https://cdn-images-1.medium.com/max/1600/1*oNt9G9UpVhtyFLDBwEMf8Q.png)

Density Based Clustering Animation:

[DB Scan Animation](https://www.youtube.com/watch?v=h53WMIImUuc)

Tips  
* Don't Get Overwhelmed! Some people spend their entire careers researching new clustering methods and improvements.

* Don't be a perfectionist! There are too many techniques to master, you can't learn all of them in 9 months.

* Focus on learning within the context of a problem you want to solve or a project that you are passionate about building

### 1.3 Clustering 

Clustering falls into the category of unsupervised learning. This is because there is nothing in our training data that designates the correct cluster that a data point should belong to beforehand. In fact, there's not even a "correct" _**number**_ of clusters to assign our points to. We will discuss some heuristics for choosing an **appropriate** number of clusters, but this (as in much of data science) is an area where there is no cut and dry right and wrong answer. 

Remember: "All models are wrong, but some models are useful." Data science is all about acknowledging where your model might be wrong while still pursuing something useful. 

**Why Clustering?**

Clustering answers questions about how similar or dissimilar our "data objects" are. Clustering is one of the most effective methods for summarizing datasets with this question in mind. Clustering can be thought of as a sort of "unsupervised classification." You will likely never deploy a clustering model to a production environment, they're too unreliable. Clustering is more useful as a tool for data exploration than a model for making predictions. 

“Clustering isn’t hard—it’s either easy, or not interesting”

If a good clustering exists, then it usually can be efficiently found. Clustering is the most difficult when clear clusters don't exist in the first place. In that case you should question whether or not clustering is the most appropriate or useful method. 

The purpose of clustering is to group data points that are similar along certain specified dimensions (attributes). "Similarity" is defined as the points being close together in some n-dimensional space. 

The greater the number of dimensions, the more difficult clustering becomes because the increase in dimensions makes all points this is because measures of distance are used to determine similarity between datapoints, and the greater the dimensionality the more all points become roughly equidistant with one another. (We don't have time to go further into this or demonstrate this, but clustering suffers from performance and interpretability issues in a high number of dimensions). Some of these challenges can be rectified by choosing an appropriate measure of "distance" between data points. For example, using clustering for document analysis is still fairly effective even though the analysis is of a highly-dimenaional space. 

**Applications of Clustering**

Astronomy: There's too much data from space for us to look at each individual start and galaxy and categorize it, but we can cluster them intro groups based on their observable attributes. 

[SkyCat](http://www.eso.org/sci/observing/tools/skycat.html)

[Sloan Digital Sky Survey](https://www.sdss.org/)

Document Classification / Grouping - We'll need to study a little bit of NLP before we can get into this. 

### 1.4 Types of Clustering

**Hierarchical:**

    - Agglomerative: start with individual points and combine them into larger and larger clusters
    - Divisive: Start with one cluster and divide the points into smaller clusters.

**Point Assignment:**

    - We decide on a number of clusters out of the gate, and assign points to that number of clusters.

**Hard vs Soft Clustering**
    - Hard Clustering assigns a point to a cluster
    - Soft Clustering assigns each point a probability that it's in a given cluster.
    - We're going to only deal with hard clustering, it's the more traditional approach. 

### 1.5 Clustering Distance Measures

**Distance Measures**

Did you know that there are distance measures other than euclidean distance?

- Euclidean
- Cosine
- Jaccard
- Edit Distance
- Etc. 

Clustering traditionally uses Euclidean Distance, but this particular measure of distance breaks down in high dimensionality spaces. It's what we'll use for today. If you **LOVE**  clustering and want to put a strong focus on this area of Machine learning (at the expense of focusing strongly on others) then I would suggest further personal research into different clustering algorithms and distance measures. 

I want to reiterate that you don't have to use PCA and clustering in conjunction with each other. I think it's more common that they are not used together, but it can be useful in certain cases. We might try it today for fun and so reiterate how PCA is the preprocessing step, and K-means will be the main "Machine Learning Algorithm."


There are a lot of clustering algorithms. 

YOU DON'T NEED TO BE ABLE TO CODE ALL OF THEM FROM SCRATCH IN ORDER TO APPLY THEM OR EVEN TO UNDERSTAND THEM. FOCUS ON LEARNING THINGS WITHIN THE CONTEXT OF A PROBLEM YOU ARE TRYING TO SOLVE AND ONLY LEARN THOSE THINGS THAT WILL HELP YOU SOLVE THE PROBLEM. 

# Part 2. K-Means Clustering

### 2.1 The Process

Given a set of points in n-dimensional space we want to :

1) select k random points to act as initial centroids (one point for each cluster)

2) Find the cluster of points surrounding that centroid (assign points to the centroid that they lie closest to)

3) Calculate a new centroid for the cluster

Repeat steps 2 & 3 until the model converges. (Clusters don't change)

In [0]:
# import the blob maker
from sklearn.datasets.samples_generator import make_blobs

In [0]:
#  Let's make some blobs
points, centers = make_blobs(n_samples = 100,
                    centers = 3,
                    n_features = 2)

In [0]:
# display that
print(points[:10])
print(centers[:10])

In [0]:
# Make that into a dataframe of x, y and label values
myblobs = pd.DataFrame(dict(x = points[:,0],
                       y = points[:, 1],
                       label = centers))

In [0]:
# take a look
myblobs.head()

In [0]:
# Display the clusters we made
plt.scatter(x = myblobs['x'],
            y = myblobs['y'],
            c = myblobs['label']);

**Linear Separability**
The 2D blobs below are what is called "linearly separable" Meaning that we could use straight lines to separate them with no errors. This is the most trivial case of of k-means clustering, but it will help us to demonstrate.

In [0]:
# Drop labels to prove that this is truly unsupervised learning
dfpoints = myblobs.drop('label', axis=1)
dfpoints.head()

In [0]:
# Scatter plot of our label-less data (no more colors!)

plt.scatter(x = myblobs['x'],
            y = myblobs['y']);

### 2.2. K-means clustering by hand

K-means clustering is what's known as a centroid-based clustering algorithm. A centroid is an imaginary point located at the average location of all of the points in a given cluster. For example, if I wanted to find the centroid of all of the points in the above graph I would just calculate the average of the dataset's x-coordinates to find the x value of the centroid, and the average of the dataset's y-coordinates to find the y value of the centroid.

If we plot the centroid on the graph you'll see that it lies in the middle of the points. You could imagine the centroid as if it is the center of gravity, or center of mass for a given cluster. Since in this example we're treating all of the points in the dataset as if they're in the same cluster, it will end up somewhere in the middle. We're just doing this to demonstrate what a centroid is. The K-means algorithm doesn't ever calculate the centroid for the entire dataset.

**Re-review steps of the algorithm**

Given a set of points in n-dimensional space we want to:

1) select k random points to act as initial centroids (one point for each cluster)
2) Find the cluster of points surrounding that centroid (assign points to the centroid that they lie closest to)
3) Calculate a new centroid for the cluster

Repeat steps 2 & 3 until the model converges. (Clusters don't change)

**3-means clustering**

Lets pick k=3 and start demonstrating how this algorithm actually works. 

The k-means algorithm works by picking 3 of the actual datapoints at random (in the simplest case) and treating those as the starting centroids. Using those centroids, 3 clusters are calculated.

We then use the new clusters and calculate a new centroid for each of them. Then, using those centroids we re-cluster. We perform this process over and over again until our clusters stabilize and the centroids stop moving. Lets demonstrate.

In [0]:
# Calculate the centroid of the entire dataset (only for demonstration purposes)
dataset_centroid_x = dfpoints['x'].mean()
dataset_centroid_y = dfpoints['y'].mean()

In [0]:
# what are the x and y coords of that centroid?
dataset_centroid_x, dataset_centroid_y

In [0]:
# Display the plots and their centroid
plt.scatter(x = myblobs['x'],
            y = myblobs['y'])
plt.plot(dataset_centroid_x, dataset_centroid_y, 'or')

In [0]:
# Sample random points to serve as the initial fake "centroids". These will get updated.
centroids = dfpoints.sample(3)
centroids = centroids.reset_index(drop=True)
centroids

In [0]:
# Plot initial "fake" centroids on the graph
plt.scatter(x = dfpoints['x'],
            y = dfpoints['y'])
plt.plot(dataset_centroid_x, dataset_centroid_y, 'or')

# plot the random centroids
plt.plot(centroids.iloc[:3]['x'], centroids.iloc[:3]['y'], 'xr')


In [0]:
# Let's import the usual stuff
import math
import numpy as np
from scipy.spatial import distance


In [0]:
# Calculate the Nearest Centroid to each data point
def find_nearest_centroid(df, centers, iteration):
    
    # calculate the distances between each point and each centroid
    distances = distance.cdist(df, centers, 'euclidean')

    # Get nearest centroid to each point based on distance
    nearest_centroids = np.argmin(distances, axis=1)
    # convert to a Pandas series and return values
    se = pd.Series(nearest_centroids)
    df['pass_'+iteration] = se.values
    # return entire dataframe
    return df

In [0]:
# check out our 'dfpoints' dataframe
print(dfpoints.shape)
dfpoints.head()

**first pass**

In [0]:
# Take a first pass at calculating the nearest centroid to each point
first_pass = find_nearest_centroid(dfpoints.select_dtypes(exclude='int64'), centroids, '1')

In [0]:
# reminder: where are the centroids?
centroids.head()

In [0]:
# what were the results of first pass?
first_pass.head()

In [0]:
# Display results for the first pass
def plot_clusters(dfpoints, column_header, centroids):
    colors = {0:'red', 1:'blue', 2:'gold'}
    fig, ax = plt.subplots()
    ax.plot(centroids.loc[:3, 'x'], centroids.loc[:3, 'y'], "ok")
    grouped = dfpoints.groupby(column_header)
    for key, group in grouped:
        group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key])
    plt.show()

plot_clusters(first_pass, 'pass_1', centroids)

In [0]:
# Define a function to get centroids
def get_centroids(df, column_header):
  new_centroids = df.groupby(column_header).mean()
  return new_centroids

In [0]:
# Apply the function. Have the centroids changed at all?
centroids_pass1 = get_centroids(first_pass, 'pass_1')
print(centroids)
print(centroids_pass1)

**second pass**

In [0]:
# Get Clusters for New Centroids
second_pass = find_nearest_centroid(first_pass.select_dtypes(exclude='int64'), centroids_pass1, '2')
print(second_pass.shape)
second_pass.head()

In [0]:
# Plot New Cluster

plot_clusters(second_pass, 'pass_2', centroids_pass1)

In [0]:
# Apply the function. Have the centroids changed at all?
centroids_pass2 = get_centroids(second_pass, 'pass_2')
print(centroids_pass1)
print(centroids_pass2)

**third pass**

In [0]:
# Calculate New Centroids
third_pass = find_nearest_centroid(second_pass.select_dtypes(exclude='int64'), centroids_pass2, '3')
print(third_pass.shape)
third_pass.head()



In [0]:

# Plot New Cluster
plot_clusters(third_pass, 'pass_3', centroids_pass2)

**fourth pass**

In [0]:
# Calculate New Centroids

# Get Clusters for New Centroids

# Plot New Cluster


**fifth pass**

In [0]:
# Calculate New Centroids

# Get Clusters for New Centroids

# Plot New Cluster


**sixth pass**

In [0]:
# Calculate New Centroids

# Get Clusters for New Centroids

# Plot New Cluster


**convergence**

In [0]:
# When additional passes fail to create any change, we have hit "convergence".


How many centroids == number of means (that's the K in k-means clustering). Since the centroid is the mean of a cluster, the number of centroids to choose is the most important decision to make in "k-means" clustering. The K value is the number of centroids.

Congratulations, you've just been introduced to the first method of _**picking k**_ - Just graph your points and pick a number that makes sense. This gets a lot harder once you get a dimensionality higher than 3, but... Didn't we learn about some way to take high dimensional data and turn it into 2 or 3 dimensions...?

### 2.3 K-Means Clustering with Scikit-Learn

In [0]:
# This is all a lot easier when we use a library instead of doing it by hand.
from sklearn.cluster import KMeans

In [0]:
# Instantiate the sklearn class, and pick a number of clusters.
mykmeans = KMeans(n_clusters=3)

In [0]:
# fit the instantiated model to our data.
mykmeans.fit(dfpoints)

In [0]:
# the labels are the same as the 3 centers.
mylabels = mykmeans.labels_

In [0]:
# Add our new labels to the dataframe
myblobs['predictions'] = mylabels
myblobs.head()

In [0]:
# Use our previous function to display the clusters as defined by scikit-learn.
mycentroids = get_centroids(dfpoints, 'predictions')
plot_clusters(dfpoints, 'predictions', mycentroids)

### 2.6 Important Considerations:

- Choosing the appropriate clustering method 

We've only taught you one so stick with that for today. 

- Choosing appropriate dimensions to cluster along. 

Hmmm, what would be the best dimension to cluster along? Maybe one that helps separate the clusters the best. You can do a lot of scatterplots to examine this or you could, I dunno, use a technique that maximizes the variance along certain dimensions transforming the data into principal components and then cluster along the dimensions of the principal components. 

- Choosing a distance measure

Euclidean is the most traditional, you'll learn the others if the occasion presents itself (it most likely won't) - If I'm being completely honest.

- Choosing an appropriate k (# of clusters)

THIS IS THE MOST IMPORTANT CONSIDERATION WHEN IT COMES TO K-MEANS (I mean it's in the name)

![Elbow Method](https://media.geeksforgeeks.org/wp-content/uploads/20190606105550/distortion1.png)

On the x-axis we have number of centroids (k)

On the y-axis we have "distortion" which is measured as the sum of squared distances of each point to its given cluster

Here's some code below that could be used to create a similar "Elbow" Graph.

In [0]:
sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(points)
    sum_of_squared_distances.append(km.inertia_)

In [0]:
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

Silhouette Coefficient -- measure of how far apart clusters are

high Silhouette Score = clusters are well separated
ranges from -1 to 1
The definition is a little involved, but intuitively the score is based on how much closer data points are to their own clusters than to the nearest neighbor cluster.

We can calculate it in sklearn with metrics.silhouette_score(X_scaled, labels, metric='euclidean').

https://en.wikipedia.org/wiki/Silhouette_(clustering)

In [0]:
# the metrics module will be your best friend


In [0]:
# what's our silhouette score?


### 2.4 Further Considerations

- Choosing an appropriate K

- Unlucky Initial Centroids

Unlucky Initial Centroids can 

    - result in a poor clustering
    - lead to a clustering that doesn't converge

- Computational Complexity

- What is K-means good for?

- ### Mostly Round, linearly-separable blobs.

# Part 3. Apply K-means clustering to dataset

Isotopic Composition of Plutonium Batches  
The pluton data frame has 45 rows and 4 columns, containing percentages of isotopic composition of 45 Plutonium batches.  
https://vincentarelbundock.github.io/Rdatasets/doc/cluster/pluton.html  
- Pu238: the percentages of (238)Pu, always less than 2 percent.
- Pu239: the percentages of (239)Pu, typically between 60 and 80 percent.
- Pu240: percentage of the plutonium 240 isotope.
- Pu241: percentage of the plutonium 241 isotope.

In [0]:
csv_file = "https://vincentarelbundock.github.io/Rdatasets/csv/cluster/pluton.csv"


In [0]:
# Use Pu239 and Pu240 as our features.


In [0]:
# plot them to see how they look.


### 3.1 Non-standardized

In [0]:
# Initialize an instance of the KMeans class from sklearn.


In [0]:
# Fit the instantiated model to our sliced dataframe


In [0]:
# Assign clusters back to our dataframe


In [0]:
# Get our centroids


In [0]:
# turn them into a dataframe


In [0]:
# Setup some colors for plotting


In [0]:
# Plot the scatter of our points with calculated centroids


In [0]:
# what's our silhouette score?


In [0]:
# What is the predicted centroid for each data point?  


### 3.2 K-means with PCA

In [0]:
# read back in the data, so it's fresh


In [0]:
# this time keep all 4 variables


**scale the dataset**

In [0]:
# instantiate the SKLearn class for standardization


In [0]:
# Standardize the dataset (default is normalization)


In [0]:
# Now it's a numpy array, not a dataframe


In [0]:
# Turn it back into a dataframe.


In [0]:
# So did that work? the mean should be zero


In [0]:
# and the std should be one.


**Now that we've scaled we can apply PCA**

In [0]:
# import and instantiate the PCA class


In [0]:
# Apply PCA to the data


In [0]:
#  how much variation did each principal component explain?


In [0]:
# How much total variance did we explain?


In [0]:
# How much information did we lose?


In [0]:
# Turn that into a dataframe.


**now apply clustering**

In [0]:
# Initialize an instance of the KMeans class from sklearn.


In [0]:
# Fit the instantiated model to our sliced dataframe


In [0]:
# Assign predicted clusters back to our dataframe


In [0]:
# Get our centroids


In [0]:
# turn them into a dataframe


In [0]:
# Setup some colors for plotting


In [0]:
# Plot the scatter of our points with calculated centroids


In [0]:
# what's our silhouette score?


In [0]:
# What is the predicted centroid for each data point?  


**K-Means tradeoffs**
- Unsupervised clustering model
- Iteratively finds labels given K
- Easy to implement in sklearn
- Sensitive to shape, scale of data
- Optimal K hard to evaluate

| strengths | weaknesses |
|---|---|
| K-Means is popular because it's simple and computationally efficient. | However, K-Means is highly scale dependent and isn't suitable for data of varying shapes and densities. |
| Easy to see results / intuitive. | Evaluating results is more subjective, requiring much more human evaluation than trusted metrics. |

**Additional Resources**  
- Andrew Moore's CS class at Carnegie Mellon contains good static visualization, step-by-step. His slide deck is online here: http://www.cs.cmu.edu/~cga/ai-course/kmeans.pdf. He also links to more of his tutorials on the first page.

Some helpful stackexchange questions:

- http://stats.stackexchange.com/questions/40613/why-dont-dummy-variables-have-the-continuous-adjacent-category-problem-in-clust
- http://stats.stackexchange.com/questions/174556/k-means-clustering-with-dummy-variables
- http://datascience.stackexchange.com/questions/22/k-means-clustering-for-mixed-numeric-and-categorical-data